# Classificazione di cibo
Questo notebook verrà utilizzato per addestrare un modello a riconoscere e classificare il cibo.

In [5]:
# Scarichiamo il dataset
!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/dataset_food_classification.zip
!unzip dataset_food_classification.zip -d dataset_food_classification

Archive:  dataset_food_classification.zip
   creating: dataset_food_classification/dataset/
  inflating: dataset_food_classification/__MACOSX/._dataset  
   creating: dataset_food_classification/dataset/test/
   creating: dataset_food_classification/dataset/train/
   creating: dataset_food_classification/dataset/val/
   creating: dataset_food_classification/dataset/test/Sandwich/
   creating: dataset_food_classification/dataset/test/ice_cream/
   creating: dataset_food_classification/dataset/test/chicken_curry/
   creating: dataset_food_classification/dataset/test/Donut/
   creating: dataset_food_classification/dataset/test/cheesecake/
   creating: dataset_food_classification/dataset/test/Crispy Chicken/
   creating: dataset_food_classification/dataset/test/Baked Potato/
   creating: dataset_food_classification/dataset/test/apple_pie/
   creating: dataset_food_classification/dataset/test/Fries/
   creating: dataset_food_classification/dataset/test/omelette/
   creating: dataset_food_cl

## Caricamento del dataset

In [9]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Definiamo le trasformazioni per il training e la validazione/test
# Le trasformazioni di training includono l'augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224), # Ritaglia e ridimensiona casualmente a 224x224
    transforms.RandomHorizontalFlip(), # Riflette orizzontalmente con probabilità 0.5
    transforms.RandomRotation(15),     # Ruota l'immagine di un angolo casuale tra -15 e +15 gradi
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Variazione di colore
    transforms.ToTensor(),             # Converte l'immagine PIL in un tensore PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalizzazione per modelli pre-addestrati
])

# Le trasformazioni per la validazione/test sono più semplici
val_test_transforms = transforms.Compose([
    transforms.Resize(256),            # Ridimensiona l'immagine a 256x256
    transforms.CenterCrop(224),        # Ritaglia il centro a 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Creiamo i dataset e i dataloader
train_dataset = ImageFolder(root='dataset_food_classification/dataset/train', transform=train_transforms)
val_dataset = ImageFolder(root='dataset_food_classification/dataset/val', transform=val_test_transforms)
test_dataset = ImageFolder(root='dataset_food_classification/dataset/test', transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

## Creazionde del modello

In [10]:
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

# Carichiamo un modello ResNet-18 pre-addestrato
# `weights='DEFAULT'` carica i pesi pre-addestrati su ImageNet
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Congeliamo tutti i parametri del modello pre-addestrato per preservare le feature
for param in model.parameters():
    param.requires_grad = False

# Sostituiamo il classificatore finale per il nostro numero di classi
num_classes = len(train_dataset.classes)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

# Spostiamo il modello sul dispositivo di calcolo (GPU se disponibile)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Modello ResNet-18 pronto per il transfer learning su {device}.")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/gab25/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


Modello ResNet-18 pronto per il transfer learning su cpu.


## Addestramento del modello

In [11]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

# Definiamo la loss function e l'ottimizzatore
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train() # Imposta il modello in modalità training
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() # Azzera i gradienti

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() # Calcola i gradienti
        optimizer.step() # Aggiorna i pesi

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    model.eval() # Imposta il modello in modalità valutazione
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad(): # Disabilita il calcolo dei gradienti
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

Epoch 1/10, Loss: 1.8928
Validation Loss: 1.1407, Accuracy: 65.80%
Epoch 2/10, Loss: 1.4014
Validation Loss: 0.9645, Accuracy: 69.78%
Epoch 3/10, Loss: 1.2889
Validation Loss: 0.8885, Accuracy: 71.38%
Epoch 4/10, Loss: 1.2257
Validation Loss: 0.8862, Accuracy: 71.70%
Epoch 5/10, Loss: 1.1958
Validation Loss: 0.8583, Accuracy: 72.50%
Epoch 6/10, Loss: 1.1746
Validation Loss: 0.8549, Accuracy: 72.23%
Epoch 7/10, Loss: 1.1564
Validation Loss: 0.8185, Accuracy: 72.72%
Epoch 8/10, Loss: 1.1649
Validation Loss: 0.8159, Accuracy: 72.68%
Epoch 9/10, Loss: 1.1457
Validation Loss: 0.7951, Accuracy: 74.06%
Epoch 10/10, Loss: 1.1419
Validation Loss: 0.8007, Accuracy: 73.84%


## Valutazione del modello

In [12]:
# Imposta il modello in modalità valutazione
model.eval()
test_correct = 0
test_total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f"Accuracy finale sul set di test: {test_accuracy:.2f}%")

from sklearn.metrics import classification_report
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Accuracy finale sul set di test: 72.93%
                precision    recall  f1-score   support

  Baked Potato       0.81      0.73      0.77       200
Crispy Chicken       0.70      0.84      0.76       200
         Donut       0.71      0.95      0.81       200
         Fries       0.75      0.85      0.80       200
       Hot Dog       0.72      0.74      0.73       200
      Sandwich       0.81      0.61      0.70       200
          Taco       0.61      0.49      0.54       200
       Taquito       0.64      0.69      0.66       200
     apple_pie       0.70      0.63      0.66       200
    cheesecake       0.71      0.81      0.76       200
 chicken_curry       0.75      0.76      0.75       200
     ice_cream       0.85      0.65      0.73       200
      omelette       0.67      0.73      0.70       200
         sushi       0.86      0.71      0.78       200

      accuracy                           0.73      2800
     macro avg       0.73      0.73      0.73      2800
  weig